In [137]:
# libraries
from dotenv import load_dotenv
import os

# load environment variables from .env file
_ = load_dotenv()
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage

In [138]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: list[str]
    revision_number: int
    max_revisions: int

In [139]:
model = ChatOllama(
    model="qwen2.5:7b",
    temperature=0
)

In [140]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high leveloutline of an essay. Write such an outline for the user \
provided topic. Give an outline of the essay along with any relevant notes or instructions for the sections."""

In [141]:
WRITTER_PROMPT = """You are an essay assistant tasked with writing excell Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous essay. \ 
Utilize all the information below as needed:

------

{content}"""

In [142]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission.\ 
Generate critique and recommentdations for the user's submission. \ 
provide detailed recommendations, including requests for length, depth, and clarity. """

In [143]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information \
that can be used when writing the following essay. \
Generate a list of search queries that will gather any \
relevant information. Only generate 3 queries max."""

In [144]:
RESEARCH_CRITIQUE_PROMPT = """You are a research assistant helping to improve an academic essay.
I will provide you with critical feedback about an essay. Your task is to generate specific research queries 
that will help address the weaknesses mentioned in the critique.

INSTRUCTIONS:
1. Analyze the critique carefully to identify knowledge gaps and weaknesses
2. Generate 3-5 specific search queries that would help address these issues
3. Focus on factual information, statistics, examples, or counter-arguments needed
4. Make each query concise (5-10 words) but specific enough for search engines
5. Return ONLY a JSON-compatible structure with a 'queries' list containing the search queries

OUTPUT FORMAT EXAMPLE:
{ "queries": [ "machine learning healthcare applications statistics", "ai ethics case studies 2023", "deep learning vs traditional algorithms comparison", "neural network training dataset size requirements" ] }" \
"" \
"
DO NOT include any explanations, introductions or notes outside the JSON structure.
If you're uncertain about what research is needed based on the critique, focus on the 
main topic of the essay combined with factual research terms like "statistics", 
"examples", "case studies", "comparison", etc.

NOW, based on the following critique, generate appropriate research queries:

CRITIQUE:
{{critique}}
"""

In [145]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: list[str]

我们将导入Tavily client而不是tool，因为我们在使用一些非传统方式使用它。

In [146]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

这个`node`将接收状态，然后创建一个消息列表。其中一个将是`PLAN_PROMPT`，那将是`SystemMessage`。然后创建`HumanMessage`，传入我们要做的`task`。然后我们将得到一个`response`，我们将取得这个消息的信息将其设置为`plan`。

In [147]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content=state["task"])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

这个`node`是接收`plan`并进行一些`research`。首先会生成一些`query`，这基本上是在说，我们将要调用它的响应将是我们之前调用的pydantic object，其中包含查询列表。所以我们将在消息列表上调用invoke。我们拿到了`RESEARCH_PLAN_PROMPT`，然后创建`HumanMessage`，传入我们要做的`task`。我们将会获取我们当前文档的列表，我们将用他们来撰写这篇essay。然后我们将循环遍历`query`，并在travily中进行搜索，我们取得`result`后，会将其附加到`content`中。

In [148]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state["task"])
    ])
    content = state.get('content', [])  # 使用get方法，如果键不存在则返回空列表
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

对于生成节点，我们要做的第一件事是准备内容。然后我们将创建`user_message`，我们将`plan`和`task`结合起来。然后创建一个消息列表，首先是一个带有写作提示的`SystemMessage`我们在其中格式化`content`。我们把信息传递给消息队列，得到一个响应。然后我们会更新修订号，这可以跟踪我们做了多少次修订。

In [149]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\n Here is my plan:\n\n {state['plan']}")
    messages = [
        SystemMessage(
            content=WRITTER_PROMPT.format(content=content)
        ),
        user_message
    ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }

生成之后，我们现在需要一个反思节点。反思节点会将反思提示作为系统提示，然后会提取`draft`。

In [150]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT),
        HumanMessage(content=state["draft"])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

通过`with_structured_output(Queries)`方法将模型输出强制转换为预定义的`Queries`结构体，确保输出格式规范。然后，系统将批评内容作为输入，让AI分析论文的弱点并生成针对性查询。

In [151]:
# def research_critique_node(state: AgentState):
#     queries = model.with_structured_output(Queries).invoke([
#         SystemMessage(content=RESERCH_CRITIQUE_PROMPT),
#         HumanMessage(content=state["critique"])
#     ])
#     content = state.get('content', [])  # 使用get方法，避免KeyError
#     for q in queries.queries:
#         response = tavily.search(query=q, max_results=2)
#         for r in response['results']:
#             content.append(r['content'])
#     return {"content": content}
def research_critique_node(state: AgentState):
    try:
        # 尝试获取结构化输出
        queries = model.with_structured_output(Queries).invoke([
            SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
            HumanMessage(content=state["critique"])
        ])
        
        # 检查queries是否为None或不具有预期结构
        if queries is None or not hasattr(queries, 'queries') or not queries.queries:
            # 创建基于文章主题的默认查询
            print("警告: 结构化输出生成失败，使用默认查询")
            default_queries = [
                "Messi Ronaldo comparison statistics", 
                "Best soccer player achievements", 
                "Football player career highlights"
            ]
            
            content = state.get('content', [])
            # 使用默认查询
            for q in default_queries:
                response = tavily.search(query=q, max_results=2)
                for r in response['results']:
                    content.append(r['content'])
                    
            return {"content": content}
        
        # 如果queries结构正确，按原计划继续
        content = state.get('content', [])
        for q in queries.queries:
            response = tavily.search(query=q, max_results=2)
            for r in response['results']:
                content.append(r['content'])
                
        return {"content": content}
        
    except Exception as e:
        # 记录错误
        print(f"research_critique_node中出现错误: {e}")
        
        # 错误发生时返回现有内容
        return {"content": state.get('content', [])}

这将查看修订号，如果大于最大修订次数，我们将结束，否则返回`"reflect"`继续反思。注意这是在生成步骤后的操作，我们要么完成，要么进入评论循环。

In [152]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

首先初始化图

In [153]:
builder = StateGraph(AgentState)

In [154]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [155]:
builder.set_entry_point("planner")  # 设置一个入口

In [156]:
# 条件边
builder.add_conditional_edges(
    "generate",
    should_continue,
    {END: END, "reflect": "reflect"}
)

In [157]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [ ]:
thread = {"configurable": {"thread_id": "1"}}

with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    graph = builder.compile(
        checkpointer=checkpointer,
        interrupt_after=['planner', 'generate', 'reflect', 'research_plan', 'research_critique']
    )
    for s in graph.stream({
        'task': "What different between Langchain and langsmith?",
        "max_revisions": 2,
        "revision_number": 1
    }, thread):
        print(s)
        


{'planner': {'plan': '### High-Level Outline: What is the Difference Between Langchain and LangSmith?\n\n#### Introduction\n- **Introduction to AI Tools**: Briefly introduce the context of AI tools in software development.\n- **Purpose of the Essay**: Explain that this essay aims to highlight the differences between two specific AI tools, Langchain and LangSmith.\n\n#### Section 1: Overview of Langchain\n- **Definition and Purpose**: Define what Langchain is and its primary purpose or function.\n- **Key Features**:\n  - List key features such as integration capabilities, API structure, and any unique selling points.\n- **Use Cases**: Provide examples of how Langchain can be used in real-world scenarios.\n\n#### Section 2: Overview of LangSmith\n- **Definition and Purpose**: Define what LangSmith is and its primary purpose or function.\n- **Key Features**:\n  - List key features such as integration capabilities, API structure, and any unique selling points.\n- **Use Cases**: Provide exa